https://stevesie.com/apps/airbnb-api/listings

In [1]:
import requests
import json

In [30]:
url = "https://api.airbnb.com/v2/explore_tabs"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    '_format':'for_explore_search_web',
    
    'currency':'KRW',
    'location':'서울',
    'tab_id':'home_tab',
    'items_per_grid':'306', # 최대 306 -> 수정할방법찾아야
#     'section_offset':'1',
#     'supports_for_you_v3':'true',
#     'timezone_offset':'300',
    'version':'1.3.9'
    }

In [31]:
resp = requests.get(url,params=params)

JSON 개념 정리

https://github.com/chankoo/python-for-data/blob/master/3.2%20JSON%2C%20XML.ipynb

In [32]:
res_jsn = resp.json() # .json() 메소드: response의 형태가 json 일경우 이 response를 파이썬 dict로 바로 매핑해준다

In [33]:
# with open('home_jsn_306.json', 'w') as fp: # 파일로 저장하자 -> review 크롤러에서 가져다 쓰기 위해서이다
#     json.dump(res_jsn,fp) # 파이썬 dict를 다시 json으로 변환하려면 json 모듈의 dump를 이용

__데이터 구조 개괄__
- res_jsn: {explore_tabs, metadata}
    -  __explore_tabs[0]__: { 'tab_id', 'tab_name', 'pagination_metadata', 'sections', 'experiments_metadata', 'home_tab_metadata'}
        - tab_id: home_tab
        - tab_name: homes
        - pagination_metadata: {'has_next_page', 'items_offset', 'section_offset', 'search_session_id'}
        - __sections: [0:not paginated, 1: paginated]__
            - sections[0]: ...'messages': [{'icon': 'icon-p2-aggregate-reviews',
                                           'style': 'MESSAGE_WITH_ICON',
                                       'title': 'Over 540,000 guest reviews for Seoul homes, with an average of 4.7 out of 5 stars.'}]...
                                       
            - __sections__[1]:{'backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'title', 'is_paginated', 'bankai_section_id', 'refinements', __'listings'__, 'review_items', 'breadcrumbs', 'section_metadata', 'nearby_locations', 'localized_listing_count'}
                - ...
                - __listings: [#items_per_grid \* { {listing}, {pricing_quote}, {verified}, {verified_card} }]__
                    - __listing : 해당 숙소에 대한 50가지 features__
                - ...
        - experiments_metadata
        - home_tab_metadata: {'urgency_commitment', 'golden_ticket_urgency_commitment', 'listing_cards_urgency_commitment_metadata', 'listing_cards_price_line_urgency_commitment_metadata', 'messages', 'facets', 'overrides', 'search_feed_items', __'remarketing_ids'__, 'location', 'breadcrumbs', 'listings_count', 'search', 'geography', 'price_histogram', 'filters'}
            - __remarketing_ids__ : listing_id 담은 리스트 -> 숙소 id 바로뽑을 수 있다
  
    
    
    - metadata

In [5]:
res_jsn.keys() # 가장 큰 구조, 'explore_tabs'과 'metadata'로 나뉨

dict_keys(['explore_tabs', 'metadata'])

In [6]:
res_jsn['explore_tabs'][0].keys() # explore_tabs을 살펴보자

dict_keys(['tab_id', 'tab_name', 'pagination_metadata', 'sections', 'experiments_metadata', 'home_tab_metadata'])

In [7]:
res_jsn['explore_tabs'][0]['tab_id']
res_jsn['explore_tabs'][0]['tab_name'] # 웹에서 보고있는 탭이 homes 이다

'homes'

> home 탭으로 추정되는 화면
>![2018-11-04 13-38-47](https://user-images.githubusercontent.com/38183218/47960055-11210580-e037-11e8-865d-8f3fda99ed7b.png)

계속 살펴보자

In [8]:
# pagination 즉, 페이지가 넘어가는 것에 대한 메타데이터가 담겨있음
res_jsn['explore_tabs'][0]['pagination_metadata'] # items_per_grid 300개 이상 설정시 {'has_next_page': False} 가 떠버림

{'has_next_page': True,
 'items_offset': 100,
 'section_offset': 5,
 'search_session_id': '3cb445a2-32c0-420a-91df-b38e1c87d51e'}

In [9]:
# 본격적인 컨텐츠를 구성하는 sections[1] 의 구조
res_jsn['explore_tabs'][0]['sections'][1].keys()

dict_keys(['backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'title', 'is_paginated', 'bankai_section_id', 'refinements', 'listings', 'review_items', 'breadcrumbs', 'section_metadata', 'nearby_locations', 'localized_listing_count'])

In [10]:
# listings에 숙소가 리스팅되어있다
res_jsn['explore_tabs'][0]['sections'][1]['listings'][1].keys()

dict_keys(['listing', 'pricing_quote', 'verified', 'verified_card'])

위 4가지 key중 listing에 대부분의 feature가 들어있음 (50개)

__listing dict의 key 설명__

- badges: ?
- bathroom_label, bathrooms: 화장실 수
- bed_label, bedroom_label, bedrooms: 침대 수, 침실 형태, 침실 수
- city: 표기 혼재 ex) '서울특별시', '마포구', 'Mapo-gu'
- guest_label: 수용인원?
- host_languages: 호스트 가능 언어
- host_thumbnail_url_small, host_thumbnail_url: 호스트 썸네일
- __id : 숙소 id__ 인듯?
- is_business_travel_ready: 비즈니스 트래블위한 숙소인지
- is_fully_refundable: 환불가능여부
- is_host_highly_rated: 호스트 신뢰가능 여부

... : 기타 features 생략

- kicker_content: 숙소형태 요약?
- lat, lng: 위도 경도
- localized_city: 군,구 수준 주소?
- localized_neighborhood: 지하철역 수준 주소?
- name: 숙소이름
- neighborhood: 근처 뭐있는지?
- person_capacity: 수용가능인원 (guest_label과 차이는뭐지??)
- picture_count,picture_url,picture_urls,picture: 숙소 사진 관련
- preview_amenities: 프리뷰에 보여지는 amenities들인듯 ex) 와이파이, 주방
- property_type_id: room_and_property_type 관련 id인듯
- __reviews_count: 리뷰수__
- room_type_category, room_type: 숙소형태 ex)Entire home/apt

... 기타

- star_rating: 숙소 별점
- tier_id: ?
- __user: 호스트 관련 정보(id, 썸네일..)__
- public_address: 군,구 수준까지 주소
- seo_reviews: ?
- amenity_ids, preview_amenity_names: 보유한 amenity들??
- reviews: ??


In [11]:
# listing의 키밸류 쌍을 뽑아보자
for k,v in res_jsn['explore_tabs'][0]['sections'][1]['listings'][10]['listing'].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력


||                         badges || : []
||                 bathroom_label || : 1 bath
||                      bathrooms || : 1.0
||                      bed_label || : 2 beds
||                  bedroom_label || : 2 bedrooms
||                       bedrooms || : 2
||                           beds || : 2
||                           city || : Yongsan-gu
||                    guest_label || : 4 guests
||                 host_languages || : ['en', 'ko']
||       host_thumbnail_url_small || : https://a0.muscache.com/im/pictures/970eb53a-a8bd-4a04-a6dc-861846582771.jpg?aki_policy=profile_small
||             host_thumbnail_url || : https://a0.muscache.com/im/pictures/970eb53a-a8bd-4a04-a6dc-861846582771.jpg?aki_policy=profile_x_medium
||                             id || : 16683321
||       is_business_travel_ready || : False
||            is_fully_refundable || : True
||           is_host_highly_rated || : True
||                 is_new_listing || : False
||                  is_rebooka

In [12]:
res_jsn['explore_tabs'][0]['experiments_metadata'] # 뭐없다

[]

home_tab에 대한 메타데이터도 뽑아보자

결과적으로 여기의 remarketing_ids 에서 숙소 id를 바로 뽑는다

In [13]:
for k,v in res_jsn['explore_tabs'][0]['home_tab_metadata'].items():
    if k == 'filters': continue # filters 너무 많아 출력 안하겠다
    print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||             urgency_commitment || : {'message_type': 'empty_message', 'message': {}}
|| golden_ticket_urgency_commitment || : {'message_type': 'empty_message', 'message': {}}
|| listing_cards_urgency_commitment_metadata || : []
|| listing_cards_price_line_urgency_commitment_metadata || : []
||                       messages || : {}
||                         facets || : {}
||                      overrides || : {}
||              search_feed_items || : []
||                remarketing_ids || : [8213750, 11444903, 18956008, 12778317, 22320269, 4619104, 19777144, 17560209, 16683321, 16965764, 22133272, 18739581, 18967029, 20719576, 19634147, 20129439, 1753246, 17578320, 10044927, 21007367, 12857880, 14440359, 20763289, 15596154, 18385571, 4258317, 14069246, 19378056, 17481718, 15649684, 9710807, 8996724, 22056285, 20292369, 19576908, 18473432, 1077893, 16013768, 17660903, 14885947, 11802302, 5714483, 19433253, 10127540, 9715695, 7128212, 1634115, 982772, 24129062, 14482939, 20546115, 

In [14]:
# res_jsn['explore_tabs'][0]['home_tab_metadata']['filters'] # 필터 정보 들여다볼 필요있으면 켜보자

In [15]:
res_jsn['metadata'] # 마지막으로 res_json 전체에 대한 metadata이다

{'current_tab_id': 'home_tab',
 'request_uuid': '44734ba5-83ed-4b45-8575-dca541f1e5ee',
 'search_id': '44734ba5-83ed-4b45-8575-dca541f1e5ee',
 'query': '서울',
 'query_text': '서울특별시 · Homes',
 'current_refinement_paths': ['/homes'],
 'federated_search_id': '7ac2d6a3-e859-4cf1-aa51-1fe67d422f33',
 'federated_search_session_id': '1952caa0-a801-44bb-a9e7-cb7aabcaff7c',
 'suggestions': [{'title': 'All',
   'type': 'for_you',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/for_you'],
    'refinement_path': '/for_you',
    'tab_id': 'all_tab'},
   'is_selected': False},
  {'title': 'Homes',
   'type': 'homes',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/homes'],
    'refinement_path': '/homes',
    'tab_id': 'home_tab'},
   'is_selected': True},
  {'title': 'Experiences',
   'type': 'experiences',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/experiences'],
    'refinement_path': '/experien

In [ ]:
home = {}
except_cnt = 0
while int(params['price_max']) < 1000000:
    print(params['price_min'],params['price_max'])
    
    retry = 3 # request 예외발생시 재시도 횟수
    while retry !=0:
        try:
            resp = requests.get(url,params=params)
            res_jsn = resp.json()
            home_count = len(res_jsn['explore_tabs'][0]['sections'][1]['listings'])
            break

        except Exception as e:
            retry -= 1
            print('ERR: btw {} and {}'.format(params['price_min'],params['price_max']),end='|')
            print(e)
            time.sleep(1)
            continue
    
    if retry ==0:
        print('해당 데이터 못가져옴')
        except_cnt += 1
        continue
    if home_count > 0:
        print('해당가격대 숙소 count: ',home_count)
        for el in res_jsn['explore_tabs'][0]['sections'][1]['listings']:
            tmp_dict = el['listing']
            tmp_id = str(tmp_dict['id'])
#             if home.get(tmp_id) is not None: print('home_id {} is overlapped, be updated'.format(tmp_id))
            del tmp_dict['id']
            home[tmp_id] = tmp_dict
    
    if int(params['price_max']) < 100000: # 십만원 기준으로 그미만은 100원단위, 이상은 1000원 단위로 request
        params['price_min'] = str(int(params['price_min']) + 100) # 가격필터 변경(100원단위)
        params['price_max'] = str(int(params['price_min']) + 99)
    else:
        params['price_min'] = str(int(params['price_min']) + 1000) # 가격필터 변경(1000원단위)
        params['price_max'] = str(int(params['price_min']) + 999)


In [ ]:
with open('home_jsn_14610.json','w') as fp:
    json.dump(home,fp)